In [1]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from transformers import BertTokenizer

In [2]:
df=pd.read_excel('dataset_filledsupplier_currency_orderday.xlsx')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24621 entries, 0 to 24620
Data columns (total 32 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   청구서번호        24621 non-null  object 
 1   No.          24621 non-null  int64  
 2   Subject      24599 non-null  object 
 3   Machinery    24621 non-null  object 
 4   Assembly     24621 non-null  object 
 5   청구품목         24621 non-null  object 
 6   Unnamed: 6   0 non-null      float64
 7   Part No.1    24602 non-null  object 
 8   Part No.2    3592 non-null   object 
 9   청구량          24517 non-null  float64
 10  견적           24171 non-null  object 
 11  견적수량         24517 non-null  float64
 12  견적화폐         24621 non-null  object 
 13  견적단가         24621 non-null  float64
 14  발주번호         24621 non-null  object 
 15  발주처          24621 non-null  object 
 16  발주           24621 non-null  object 
 17  발주수량         24621 non-null  int64  
 18  발주금액         24621 non-null  float64
 19  D/T 

In [4]:
missing_conditions = df[
    df['발주'].notnull() &  # 발주 일자는 비어있지 않음
    df['미입고 기간'].isnull() &  # 미입고 기간은 비어있음
    df['창고입고'].isnull() & # 창고 입고도 비어있음
    df['선박입고'].isnull()  # 선박 입고도 비어있음

]

print(f"발주 일자는 있지만 미입고 기간, 창고 입고, 선박 입고도 없는 경우: {len(missing_conditions)}개")
# 해당 조건의 행 삭제
df = df.drop(missing_conditions.index)

print(f"삭제된 행의 개수: {len(missing_conditions)}개")
print(f"남은 데이터프레임의 크기: {df.shape}")

발주 일자는 있지만 미입고 기간, 창고 입고, 선박 입고도 없는 경우: 1699개
삭제된 행의 개수: 1699개
남은 데이터프레임의 크기: (22922, 32)


In [5]:
#미입고기간으로 처리.
missing_both = df[df['창고입고'].isnull() & df['미입고 기간'].notnull()]

print(f"창고 입고일은 없고 미입고 기간은 명시되어 있어 미입고 기간으로 분류해야 할 경우 : {len(missing_both)}개")

창고 입고일은 없고 미입고 기간은 명시되어 있어 미입고 기간으로 분류해야 할 경우 : 1620개


# 리드타임 계산

> 1. 발주와 입고가 같은 경우 리드타임을 1로 설정
> 2. 발주 수량과 입고 수량이 같은 경우, 일반 계산
> 3. 발주 수량과 입고 수량이 다른 경우에는 가중 평균을 리드타임을 계산

In [6]:
df['발주'] = pd.to_datetime(df['발주'])
df['창고입고'] = pd.to_datetime(df['창고입고'])

same_date_df = df[df['발주'] == df['창고입고']]

# 필터링된 행의 수를 세기
count_same_date = same_date_df.shape[0]

print(f"창고입고와 발주 날짜가 같은 행의 개수: {count_same_date}")

창고입고와 발주 날짜가 같은 행의 개수: 292


In [7]:
df['발주'] = pd.to_datetime(df['발주'], errors='coerce')
df['창고입고'] = pd.to_datetime(df['창고입고'], errors='coerce')

# 미입고기간 처리
def convert_leadtime(기간):
    if 기간 == '1개월':
        return 1001
    elif 기간 == '3개월':
        return 1002
    elif 기간 == '6개월':
        return 1003
    elif 기간 == '6개월 초과':
        return 1004
    else:
        return None 

df['delay'] = df['미입고 기간'].apply(convert_leadtime)

# 리드타임 계산 
def calculate_leadtime(row):
    if pd.isna(row['창고입고']):
        return None  # 입고가 없는 경우
    elif row['발주'] == row['창고입고']:
        return 1  # 발주와 입고가 같은 경우 리드타임 1
    else:
        return (row['창고입고'] - row['발주']).days 

# 미입고기간이 없는 경우 리드타임 계산
df['leadtime'] = df.apply(lambda row: calculate_leadtime(row) if pd.isna(row['미입고 기간']) else None, axis=1)

# 가중리드타임 초기화
df['weighted_leadtime'] = None

# 미입고기간이 없는 데이터에 대해 가중 평균 리드타임 계산
df_filtered = df[df['미입고 기간'].isna()]
grouped = df_filtered.groupby(['발주처', '발주', '발주수량', '발주금액'])

for name, group in grouped:
    if len(group) > 1 and group['창고입고수량'].nunique() > 1:
        weighted_leadtime = (group['leadtime'] * group['창고입고수량']).sum() / group['창고입고수량'].sum()
        df.loc[group.index, 'weighted_leadtime'] = weighted_leadtime


In [8]:
df['weighted_leadtime'].notnull().sum()

60

In [9]:
# 'target' 열 정의 (weighted_leadtime이 있으면 사용, 없으면 leadtime 사용)
df['target'] = df['weighted_leadtime'].combine_first(df['leadtime'])
df['y'] = df['target'].combine_first(df['delay'])  # 미입고 기간 값도 함께 결합


In [10]:
# 'y' 열에 결측값이 있는지 확인
print(df['y'].isnull().sum())  # 결측값이 몇 개인지 확인

# 'y' 열이 잘 생성되었는지 데이터프레임 확인
print(df[['target', 'delay', 'y']].head())

0
  target  delay      y
0  112.0    NaN  112.0
1   97.0    NaN   97.0
2  112.0    NaN  112.0
3    1.0    NaN    1.0
4    1.0    NaN    1.0


> XGBRegressor로 특성 중요도 분석 결과 > 견적화폐 0.6 이상

# 다중 출력 모델
### 리드타임 예측 (회귀) / 미입고 기간 예측 (분류) 
1. 텍스트 칼럼 결합 및 BERT 임베딩
2. ( 수치형 데이터(견적단가 및 발주량) Scaling )
3. 범주형 데이터(견적화폐) onehotEncoding
4. BERT 임베딩 유사도 => 모델의 입력, 2.3데이터 결합 => 리드타임 OR 미입고 기간 예측

### 전처리

In [11]:
import re

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'\([^)]*\)', '', text)
    text = re.sub(r'[^\w\s\*/\-\+.,#&]', '', text)
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'\b(사용금지|사)\b', '', text, flags=re.IGNORECASE)
    text = text.strip()
    return text

def clean_supplier_name(name):
    name = name.lower()
    name = re.sub(r'coporation|coropration|coproration|corporration', 'corporation', name)
    name = re.sub(r'\(사용금지\)', '', name)
    name = re.sub(r'u\.s\.a', '_usa', name)
    name = re.sub(r'\.', '', name)
    suffixes = r'(corporation|corp|company|co|incorporated|inc|limited|ltd|상사|공사|엔지니어링|주식회사|주|gmbh|pte ltd|llc)'
    name = re.sub(suffixes, '', name, flags=re.IGNORECASE)
    name = re.sub(r'[^\w\s-]', '', name)
    name = re.sub(r'\s+', ' ', name).strip()
    return name

In [12]:
# 텍스트 컬럼 리스트
text_columns = ['Machinery', 'Assembly', '청구품목', 'Part No.1', '발주처']

for col in text_columns:
    df[col] = df[col].astype(str)
df['cleaned_machinery'] = df['Machinery'].apply(preprocess_text)
df['cleaned_assembly'] = df['Assembly'].apply(preprocess_text)
df['cleaned_item'] = df['청구품목'].apply(preprocess_text)
df['cleaned_supplier'] = df['발주처'].apply(clean_supplier_name)

df['combined_text'] = (
    df['cleaned_machinery'].fillna('') + " " +
    df['cleaned_assembly'].fillna('') + " " +
    df['cleaned_item'].fillna('') + " " +
    df['Part No.1'].fillna('') + " " +
    df['cleaned_supplier'].fillna('')
)

In [13]:
# 최종 데이터프레임 선택
final_df = df[['Machinery', 'Assembly', '청구품목', 'Part No.1', '발주처', '견적화폐', 
               '발주수량', '발주금액', '미입고 기간', '창고입고', 'leadtime', 
               'weighted_leadtime', 'delay', 'combined_text', 'target', 'y']].copy()

# 최종 데이터프레임 확인
print(final_df.shape)
print(final_df.head())

(22922, 16)
                          Machinery       Assembly  \
0    CARGO BOOM VANG BLOCK (STBD 하)          BLOCK   
1  SPANISH BOOM VANG BLOCK (PORT 상)          BLOCK   
2                       PURSE BLOCK      TOW BLOCK   
3                       MAIN ENGINE  POWER PACK AS   
4                       MAIN ENGINE  POWER PACK AS   

                                               청구품목               Part No.1  \
0  MCKISSICK CONSTRUCTION BLOCKS (WIRE SIZE : 5/8")                C15S10BS   
1  MCKISSICK CONSTRUCTION BLOCKS (WIRE SIZE : 5/8")                C15D10BS   
2                            WESTEC 20TON TOW BLOCK  WESTEC 20TON TOW BLOCK   
3                        GE POWER PACK FORK - E7(B)                40028340   
4                        GE POWER PACK FORK - E7(B)                40028340   

                         발주처 견적화폐  발주수량          발주금액 미입고 기간       창고입고  \
0  MATSUI(U.S.A) COROPRATION  USD     2  2.288732e+06    NaN 2019-05-03   
1  MATSUI(U.S.A) COROPRATION  USD     

In [14]:
# OneHotEncoder 초기화 및 변환 (final_df에 적용)
currency_ohe = OneHotEncoder(sparse_output=False)
currency_encoded = currency_ohe.fit_transform(final_df[['견적화폐']])
currency_encoded_df = pd.DataFrame(currency_encoded, columns=currency_ohe.get_feature_names_out(['견적화폐']))

# 원본 데이터프레임에 원-핫 인코딩된 화폐 정보 추가
data = pd.concat([final_df.reset_index(drop=True), currency_encoded_df.reset_index(drop=True)], axis=1)


In [15]:
print(f"Final_df shape: {final_df.shape}")
print(f"Currency_encoded_df shape: {currency_encoded_df.shape}")
print(f"Data shape after concat: {data.shape}")

Final_df shape: (22922, 16)
Currency_encoded_df shape: (22922, 4)
Data shape after concat: (22922, 20)


In [16]:
data['leadtime'].notnull().sum()

21302

In [17]:
data['미입고 기간'].notnull().sum()

1620

In [18]:
data['y'].isnull().sum()

0

### 데이터 분할

> 리드타임 예측 데이터와 미입고 기간 분류 데이터로 나누어서 각각 학습

In [19]:
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertModel
import torch
from torch.utils.data import DataLoader, TensorDataset 

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [20]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased').to(device)

C:\Users\User\anaconda3\envs\ship\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
C:\Users\User\anaconda3\envs\ship\Lib\site-packages\torch\nn\modules\module.py:1160: UserWarning: expandable_segments not supported on this platform (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\c10/cuda/CUDAAllocatorConfig.h:28.)
  return t.to(


In [21]:
def encode_data(texts):
    return tokenizer(texts.tolist(), padding=True, truncation=True, max_length=128, return_tensors='pt').to(device)

In [22]:
def get_embeddings_batchwise(encodings, model, batch_size=16):
    dataset = TensorDataset(encodings['input_ids'], encodings['attention_mask'], encodings['token_type_ids'])
    dataloader = DataLoader(dataset, batch_size=batch_size)

    all_embeddings = []
    model.eval()
    with torch.no_grad():
        for batch in dataloader:
            input_ids, attention_mask, token_type_ids = [t.to(device) for t in batch]
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
            all_embeddings.append(outputs.pooler_output.cpu())

    return torch.cat(all_embeddings, dim=0)

In [23]:


# 리드타임 예측용 데이터
train_leadtime = data[~data['target'].isnull()]
X_leadtime_text = train_leadtime['combined_text']
X_leadtime_features = train_leadtime[currency_encoded_df.columns]
y_leadtime = train_leadtime['target']

# 미입고 기간 분류용 데이터
train_delay = data[~data['delay'].isnull()]
X_delay_text = train_delay['combined_text']
X_delay_features = train_delay[currency_encoded_df.columns]
y_delay = train_delay['delay']

# 리드타임용 train/val/test 분할
X_train_leadtime_text, X_temp_leadtime_text, X_train_leadtime_features, X_temp_leadtime_features, y_train_leadtime, y_temp_leadtime = train_test_split(
    X_leadtime_text, X_leadtime_features, y_leadtime, test_size=0.2, random_state=42
)
X_val_leadtime_text, X_test_leadtime_text, X_val_leadtime_features, X_test_leadtime_features, y_val_leadtime, y_test_leadtime = train_test_split(
    X_temp_leadtime_text, X_temp_leadtime_features, y_temp_leadtime, test_size=0.5, random_state=42
)

# 미입고 기간용 train/val/test 분할
X_train_delay_text, X_temp_delay_text, X_train_delay_features, X_temp_delay_features, y_train_delay, y_temp_delay = train_test_split(
    X_delay_text, X_delay_features, y_delay, test_size=0.2, random_state=42
)
X_val_delay_text, X_test_delay_text, X_val_delay_features, X_test_delay_features, y_val_delay, y_test_delay = train_test_split(
    X_temp_delay_text, X_temp_delay_features, y_temp_delay, test_size=0.5, random_state=42
)

In [24]:
print("리드타임 예측용 데이터에서 null 값 확인:")
print(f"y_leadtime null 개수: {y_leadtime.isnull().sum()}")
print("미입고 기간 분류용 데이터에서 null 값 확인:")
print(f"y_delay null 개수: {y_delay.isnull().sum()}")

리드타임 예측용 데이터에서 null 값 확인:
y_leadtime null 개수: 0
미입고 기간 분류용 데이터에서 null 값 확인:
y_delay null 개수: 0


In [25]:
print("Train/Val/Test 데이터에 null 값 확인 (리드타임 예측용):")
print(f"y_train_leadtime null 개수: {y_train_leadtime.isnull().sum()}")
print(f"y_val_leadtime null 개수: {y_val_leadtime.isnull().sum()}")
print(f"y_test_leadtime null 개수: {y_test_leadtime.isnull().sum()}")

print("Train/Val/Test 데이터에 null 값 확인 (미입고 기간 분류용):")
print(f"y_train_delay null 개수: {y_train_delay.isnull().sum()}")
print(f"y_val_delay null 개수: {y_val_delay.isnull().sum()}")
print(f"y_test_delay null 개수: {y_test_delay.isnull().sum()}")

Train/Val/Test 데이터에 null 값 확인 (리드타임 예측용):
y_train_leadtime null 개수: 0
y_val_leadtime null 개수: 0
y_test_leadtime null 개수: 0
Train/Val/Test 데이터에 null 값 확인 (미입고 기간 분류용):
y_train_delay null 개수: 0
y_val_delay null 개수: 0
y_test_delay null 개수: 0


In [26]:
# 리드타임 예측용 임베딩
train_leadtime_encodings = encode_data(X_train_leadtime_text)
val_leadtime_encodings = encode_data(X_val_leadtime_text)
test_leadtime_encodings = encode_data(X_test_leadtime_text)

train_leadtime_embeddings = get_embeddings_batchwise(train_leadtime_encodings, bert_model, batch_size=8).to(device)
val_leadtime_embeddings = get_embeddings_batchwise(val_leadtime_encodings, bert_model, batch_size=8).to(device)
test_leadtime_embeddings = get_embeddings_batchwise(test_leadtime_encodings, bert_model, batch_size=8).to(device)

# 미입고 기간 예측용 임베딩
train_delay_encodings = encode_data(X_train_delay_text)
val_delay_encodings = encode_data(X_val_delay_text)
test_delay_encodings = encode_data(X_test_delay_text)

train_delay_embeddings = get_embeddings_batchwise(train_delay_encodings, bert_model, batch_size=8).to(device)
val_delay_embeddings = get_embeddings_batchwise(val_delay_encodings, bert_model, batch_size=8).to(device)
test_delay_embeddings = get_embeddings_batchwise(test_delay_encodings, bert_model, batch_size=8).to(device)


C:\Users\User\anaconda3\envs\ship\Lib\site-packages\transformers\models\bert\modeling_bert.py:439: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


In [27]:
# 추가 피처 (화폐 정보 등)를 임베딩과 결합
train_leadtime_input = torch.cat((train_leadtime_embeddings.to(device), torch.tensor(X_train_leadtime_features.values, dtype=torch.float).to(device)), dim=1)
val_leadtime_input = torch.cat((val_leadtime_embeddings.to(device), torch.tensor(X_val_leadtime_features.values, dtype=torch.float).to(device)), dim=1)
test_leadtime_input = torch.cat((test_leadtime_embeddings.to(device), torch.tensor(X_test_leadtime_features.values, dtype=torch.float).to(device)), dim=1)

train_delay_input = torch.cat((train_delay_embeddings.to(device), torch.tensor(X_train_delay_features.values, dtype=torch.float).to(device)), dim=1)
val_delay_input = torch.cat((val_delay_embeddings.to(device), torch.tensor(X_val_delay_features.values, dtype=torch.float).to(device)), dim=1)
test_delay_input = torch.cat((test_delay_embeddings.to(device), torch.tensor(X_test_delay_features.values, dtype=torch.float).to(device)), dim=1)

In [35]:
# y_train_leadtime과 y_train_delay를 텐서로 변환
y_train_leadtime_tensor = torch.tensor(y_train_leadtime.values.astype(float), dtype=torch.float).to(device)
y_train_delay_tensor = torch.tensor(y_train_delay.values.astype(int), dtype=torch.long).to(device)

# y_val과 y_test도 마찬가지로 변환
y_val_leadtime_tensor = torch.tensor(y_val_leadtime.values.astype(float), dtype=torch.float).to(device)
y_val_delay_tensor = torch.tensor(y_val_delay.values.astype(int), dtype=torch.long).to(device)

y_test_leadtime_tensor = torch.tensor(y_test_leadtime.values.astype(float), dtype=torch.float).to(device)
y_test_delay_tensor = torch.tensor(y_test_delay.values.astype(int), dtype=torch.long).to(device)

In [42]:
print(f"Train input shape (Leadtime): {train_leadtime_input.shape}")
print(f"Train target shape (Leadtime): {y_train_leadtime_tensor.shape}")
print(f"Train input shape (Delay): {train_delay_input.shape}")
print(f"Train target shape (Delay): {y_train_delay_tensor.shape}")

Train input shape (Leadtime): torch.Size([17041, 772])
Train target shape (Leadtime): torch.Size([17041])
Train input shape (Delay): torch.Size([1296, 772])
Train target shape (Delay): torch.Size([1296])


In [43]:
import torch.nn as nn

class MultiOutputModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_classes):
        super(MultiOutputModel, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc_regression = nn.Linear(hidden_dim, 1)  # 리드타임 회귀
        self.fc_classification = nn.Linear(hidden_dim, 4)  # 미입고 기간 분류

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        regression_output = self.fc_regression(x)
        classification_output = self.fc_classification(x)
        return regression_output, classification_output


In [44]:
input_dim = train_leadtime_input.shape[1]
hidden_dim = 128
num_classes = 4

model = MultiOutputModel(input_dim=input_dim, hidden_dim=hidden_dim, num_classes=num_classes).to(device)

In [45]:
loss_fn_regression = torch.nn.MSELoss()  # 리드타임 회귀
loss_fn_classification = torch.nn.CrossEntropyLoss()  # 미입고 기간 분류

# 옵티마이저
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)

In [46]:
# 리드타임 학습
def train_leadtime_model(model, train_input, target_regression, optimizer, device):
    model.train()
    optimizer.zero_grad()

    train_input = train_input.to(device)
    target_regression = target_regression.to(device)
    
    # Forward pass
    regression_output, _ = model(train_input)  # 미입고 분류는 무시하고 리드타임만 학습

    # 손실 계산
    loss_regression = loss_fn_regression(regression_output, target_regression)
    
    # Backward pass and optimization
    loss_regression.backward()
    optimizer.step()

    return loss_regression.item()

In [47]:
# 미입고 기간 학습
def train_delay_model(model, train_input, target_classification, optimizer, device):
    model.train()
    optimizer.zero_grad()

    train_input = train_input.to(device)
    target_classification = target_classification.to(device)
    
    # Forward pass
    _, classification_output = model(train_input)  # 리드타임은 무시하고 미입고 기간만 학습

    # 손실 계산
    loss_classification = loss_fn_classification(classification_output, target_classification)
    
    # Backward pass and optimization
    loss_classification.backward()
    optimizer.step()

    return loss_classification.item()

In [48]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, accuracy_score, f1_score

def evaluate_model(model, val_input, val_target_regression, val_target_classification, device):
    model.eval()
    
    val_input = val_input.to(device)
    val_target_regression = val_target_regression.to(device)
    val_target_classification = val_target_classification.to(device)
    
    with torch.no_grad():
        regression_output, classification_output = model(val_input)
    
    # 리드타임 회귀 손실 및 평가 지표 계산
    val_loss_regression = loss_fn_regression(regression_output, val_target_regression)
    mae = mean_absolute_error(val_target_regression.cpu().numpy(), regression_output.cpu().numpy())
    rmse = mean_squared_error(val_target_regression.cpu().numpy(), regression_output.cpu().numpy(), squared=False)
    
    # 미입고 기간 분류 손실 및 평가 지표 계산
    val_loss_classification = loss_fn_classification(classification_output, val_target_classification)
    acc = accuracy_score(val_target_classification.cpu().numpy(), classification_output.argmax(dim=1).cpu().numpy())
    f1 = f1_score(val_target_classification.cpu().numpy(), classification_output.argmax(dim=1).cpu().numpy(), average='weighted')
    
    total_loss = val_loss_regression.item() + val_loss_classification.item()
    
    print(f"Validation Loss: {total_loss:.4f}, MAE: {mae:.4f}, RMSE: {rmse:.4f}, Accuracy: {acc:.4f}, F1-Score: {f1:.4f}")
    

In [49]:
# 학습 및 평가 실행
epochs = 10
for epoch in range(epochs):
    # 리드타임 데이터 학습
    train_loss_leadtime = train_leadtime_model(model, train_leadtime_input, y_train_leadtime_tensor, optimizer, device)
    
    # 미입고 기간 데이터 학습
    train_loss_delay = train_delay_model(model, train_delay_input, y_train_delay_tensor, optimizer, device)

    print(f"Epoch {epoch+1}/{epochs}, Train Loss (Leadtime): {train_loss_leadtime:.4f}, Train Loss (Delay): {train_loss_delay:.4f}")
    
    # 평가
    val_loss, val_mae, val_rmse, val_acc, val_f1 = evaluate_model(model, val_leadtime_input, y_val_leadtime_tensor, y_val_delay_tensor, device)
    
    print(f"Epoch {epoch+1}/{epochs}, Validation Loss: {val_loss:.4f}, MAE: {val_mae:.4f}, RMSE: {val_rmse:.4f}, Accuracy: {val_acc:.4f}, F1-Score: {val_f1:.4f}")

OutOfMemoryError: CUDA out of memory. Tried to allocate 1.08 GiB. GPU 0 has a total capacity of 4.00 GiB of which 0 bytes is free. Of the allocated memory 2.77 GiB is allocated by PyTorch, and 65.22 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
def predict(model, input_data, device):
    model.eval()  # 평가 모드로 설정
    with torch.no_grad():
        regression_output, classification_output = model(input_data)
    return regression_output, classification_output

In [ ]:
import torch
import torch.nn.functional as F

# 코사인 유사도 계산 함수
def cosine_similarity(embedding1, embedding2):
    return F.cosine_similarity(embedding1, embedding2, dim=1)

# 최종 예측 함수: 임베딩 유사도를 기준으로 더 신뢰할 수 있는 예측을 선택
def predict_with_embedding_similarity(model, leadtime_input, delay_input, leadtime_embedding, delay_embedding):
    # 모델을 통해 리드타임과 미입고 기간 예측
    leadtime_pred, _ = model(leadtime_input)
    _, delay_pred = model(delay_input)

    # 리드타임 예측과 미입고 기간 예측의 임베딩 유사도 계산
    leadtime_similarity = cosine_similarity(leadtime_input, leadtime_embedding)
    delay_similarity = cosine_similarity(delay_input, delay_embedding)

    # 유사도를 비교하여 더 높은 쪽의 예측을 선택
    if torch.mean(leadtime_similarity) > torch.mean(delay_similarity):
        return leadtime_pred, "Leadtime prediction"
    else:
        return delay_pred, "Delay prediction"

# 예측 예시 (train_leadtime_input, train_delay_input는 입력 데이터)
final_prediction, chosen_model = predict_with_embedding_similarity(
    model, train_leadtime_input, train_delay_input, train_leadtime_embeddings, train_delay_embeddings
)

print(f"Chosen Model: {chosen_model}")